# Cycle GAN implementation by tensorflow

In [1]:
import tensorflow as tf
import os

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

#from util import *
#from ops import *

In [2]:
# convert imgs into .npy file
def ConvertIntoNpy(src, dst):
    files = os.listdir(src)
    firstImg = Image.open(src + "/" + files[0])
    np_firstImg = np.asarray(firstImg)
    fileList = np.copy(np_firstImg)
    fileList = fileList[np.newaxis, :]

    print("num of files: " + str(len(files)))
    print("file size: " + str(np_firstImg.shape))

    for i in range(len(files)):
        if i % 100 == 1:
            print(i)

        img = Image.open(src + "/" + files[i])
        np_img = np.asarray(img)
        if np_img.shape != np_firstImg.shape:
            continue

        np_img = np_img[np.newaxis, :]
        fileList = np.vstack((fileList, np_img))
        img.close()

    print("generated npy size: " + str(fileList.shape))
    np.save(dst, fileList)

#ConvertIntoNpy("ukiyoe2photo/trainA", "./ukiyoe2photo_trainA.npy")

In [3]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss_X"], label="discriminative loss X", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "d_loss_X" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss_Y"], label="discriminative loss Y", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "d_loss_Y" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

In [4]:
# plot images
def save_imgs(images, plot_dim=(1,2), size=(8,4), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 2
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((256, 256, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

# model

In [5]:
class Generator:
    def __init__(self, name=""):
        self.reuse = False
        self.name = name
        print("init... name of g:" + str(self.name))

    def __call__(self, inputs, gf_dim=64):
        def residual_block(x, dim):
            # hxwxdim
            y = tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], "REFLECT") # (h+1)x(w+1)x(dim)
            y = tf.layers.conv2d(y, dim, [3,3], [1,1], padding="VALID") # (h)x(w)x(dim)
            y = tf.layers.batch_normalization(y)
            y = tf.nn.relu(y)
            y = tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], "REFLECT") # (h+1)x(w+1)x(dim)
            y = tf.layers.conv2d(y, dim, [3,3], [1,1], padding="VALID") # (h)x(w)x(dim)
            y = tf.layers.batch_normalization(y)
            return y + x

        with tf.variable_scope('g_' + self.name, reuse=self.reuse):
            # 256x256x3
            print("called... name of g:" + str(self.name))
            # print("inputs" + str(inputs.get_shape()))
            c0 = tf.pad(inputs, [[0,0], [3,3], [3,3], [0,0]], "REFLECT")  # (262)x(262)x3
            # print("c0" + str(c0.get_shape()))
            c1 = tf.layers.conv2d(c0, gf_dim, [7,7], [1,1], padding="VALID") # (256)x(256)xgf_dim
            c1 = tf.nn.relu(tf.layers.batch_normalization(c1))
            # print("c1" + str(c1.get_shape()))
            
            c2 = tf.layers.conv2d(c1, gf_dim*2, [3,3], [2,2], padding="SAME") # (128)x(128)x(gf_dim*2)
            c2 = tf.nn.relu(tf.layers.batch_normalization(c2)) 
            # print("c2" + str(c2.get_shape()))
            
            c3 = tf.layers.conv2d(c2, gf_dim*4, [3,3], [2,2], padding="SAME") # (64x64xgf_dim*4)
            c3 = tf.nn.relu(tf.layers.batch_normalization(c3))
            # print("c3" + str(c3.get_shape()))
            
            r1 = residual_block(c3, gf_dim*4)
            r2 = residual_block(r1, gf_dim*4)
            r3 = residual_block(r2, gf_dim*4)
            r4 = residual_block(r3, gf_dim*4)
            r5 = residual_block(r4, gf_dim*4)
            r6 = residual_block(r5, gf_dim*4)
            r7 = residual_block(r6, gf_dim*4)
            r8 = residual_block(r7, gf_dim*4)
            r9 = residual_block(r8, gf_dim*4)
            # print("r9" + str(r9.get_shape()))            
            # 64x64xgf_dim*4
            d1 = tf.layers.conv2d_transpose(r9, gf_dim*2, [3,3], [2,2], padding="SAME") # 128x128
            d1 = tf.nn.relu(tf.layers.batch_normalization(d1))
            # print("d1" + str(d1.get_shape()))     
            d2 = tf.layers.conv2d_transpose(d1, gf_dim, [3,3], [2,2], padding="SAME") # 256x256
            d2 = tf.nn.relu(tf.layers.batch_normalization(d2))
            d2 = tf.pad(d2, [[0,0], [3,3], [3,3], [0,0]], "REFLECT")
            # print("d2" + str(d2.get_shape()))                 
            y = tf.layers.conv2d(d2, 3, [7,7], [1,1], padding="VALID") # 256x256x3
            y = tf.nn.tanh(y)
            # print("y" + str(y.get_shape()))     
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return y

In [6]:
class Discriminator:
    def __init__(self, name=""):
        self.reuse = False
        self.name = name
        print("init... name of d:" + self.name)
    def __call__(self, x, df_dim=64):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.variable_scope('d_' + self.name, reuse=self.reuse):
            # x: 256x256x3
            print("called... name of d:" + self.name)
            conv1 = tf.layers.conv2d(x, df_dim, [4,4], [2,2]) # 128x128xdif_dim
            conv1 = leaky_relu(conv1)

            conv2 = tf.layers.conv2d(conv1, df_dim*2, [4,4], [2,2]) # 64x64xdf_dim*2
            conv2 = leaky_relu(conv2)

            conv3 = tf.layers.conv2d(conv2, df_dim*4, [4,4], [2,2]) # 32x32xdf_dim*4
            conv3 = leaky_relu(conv3)

            conv4 = tf.layers.conv2d(conv3, df_dim*8, [4,4], [1,1]) # 32x32xdf_dim*8
            conv4 = leaky_relu(conv4)

            conv5 = tf.layers.conv2d(conv4, 1, [4,4], [1,1]) # 32x32x1

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return conv5

In [7]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [8]:
def l2_loss(x1, x2, weights=1.0):
    loss = tf.reduce_mean((x1 - x2) ** 2) * weights
    return loss

def l1_loss(x1, x2, weights=1.0):
    loss = tf.reduce_mean(tf.abs(x1 - x2)) * weights
    return loss

In [ ]:
class cycleGAN:
    def __init__(self):
        # definition of G and D for each X2Y, Y2X
        self.g_X2Y = Generator(name="X2Y")
        self.g_Y2X = Generator(name="Y2X")
        self.d_X2Y = Discriminator(name="X2Y")
        self.d_Y2X = Discriminator(name="Y2X")
        

        self.X_train = np.load("ukiyoe2photo_testA.npy")
        self.X_train = self.X_train/255

        self.Y_train = np.load("ukiyoe2photo_testB.npy")
        self.Y_train = self.Y_train/255

        self.X_dim = 256*256*3
        self.batch_size = 2
        self.epochs = 1000000

        self.epoch_saveMetrics = 10000
        self.epoch_saveSampleImg = 500
        self.epoch_saveParamter = 10000

        self.loss = {"d_loss_X":[], "d_loss_Y":[], "g_loss":[]}
        
        self.crop_size = 256

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.crop_size, self.crop_size, 3])
        self.Y_tr = tf.placeholder(tf.float32, shape=[None, self.crop_size, self.crop_size, 3])

        # the results of generation
        self.X2Y = self.g_X2Y(self.X_tr) #X→Y
        self.Y2X = self.g_Y2X(self.Y_tr) #Y→X
#        print(Y2X.get_shape())
        self.X2Y2X = self.g_Y2X(self.X2Y) #X→Y→X
        self.Y2X2Y = self.g_X2Y(self.Y2X) #Y→X→Y
#        print(Y2X2Y.get_shape())

        self.lr = 0.0002

        
    def loss_(self):

        # the results of discrimination
        X_dis = self.d_Y2X(self.X_tr)
        Y2X_dis = self.d_Y2X(self.Y2X)
        Y_dis = self.d_X2Y(self.Y_tr)
        X2Y_dis = self.d_X2Y(self.X2Y)

        # g_losses
        g_loss_X2Y = tf.identity(l2_loss(X2Y_dis, tf.ones_like(X2Y_dis)))
        g_loss_Y2X = tf.identity(l2_loss(Y2X_dis, tf.ones_like(Y2X_dis)))
        cyc_loss_X = tf.identity(l1_loss(self.X_tr, self.X2Y2X) * 10.0)
        cyc_loss_Y = tf.identity(l1_loss(self.Y_tr, self.Y2X2Y) * 10.0)
        g_loss = g_loss_X2Y + g_loss_Y2X + cyc_loss_X + cyc_loss_Y

        #  d_losses
        d_loss_Xtr = l2_loss(X_dis, tf.ones_like(X_dis))
        d_loss_Y2X = l2_loss(Y2X_dis, tf.ones_like(Y2X_dis))
        d_loss_X = d_loss_Xtr + d_loss_Y2X
        
        d_loss_Ytr = l2_loss(Y_dis, tf.ones_like(Y_dis))
        d_loss_X2Y = l2_loss(X2Y_dis, tf.ones_like(X2Y_dis))
        d_loss_Y = d_loss_Ytr + d_loss_X2Y
 
        return g_loss, d_loss_X, d_loss_Y

    def train(self):
        g_loss, d_loss_X, d_loss_Y = self.loss_()

        # Optimizer
        g_var = [var for var in (self.g_X2Y.variables or self.g_Y2X.variables)]
        d_X_train_op = tf.train.AdamOptimizer(self.lr, beta1=0.5).minimize(d_loss_X, var_list=self.d_Y2X.variables)
        d_Y_train_op = tf.train.AdamOptimizer(self.lr, beta1=0.5).minimize(d_loss_Y, var_list=self.d_X2Y.variables)
        g_train_op = tf.train.AdamOptimizer(self.lr, beta1=0.5).minimize(g_loss, var_list=g_var)

        saver = tf.train.Saver()
        #%debug
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(self.epochs):

                # X_mb, Y_mbを収集
                rand_index = np.random.randint(0, self.X_train.shape[0], size=self.batch_size)
                X_mb = self.X_train[rand_index, :].astype(np.float32)
                rand_index = np.random.randint(0, self.Y_train.shape[0], size=self.batch_size)
                Y_mb = self.Y_train[rand_index, :].astype(np.float32)

                # train G
                _, g_loss_value = sess.run([g_train_op, g_loss], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # train D_X
                _, d_loss_X_value = sess.run([d_X_train_op, d_loss_X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # train D_Y
                _, d_loss_Y_value = sess.run([d_Y_train_op, d_loss_Y], feed_dict={self.X_tr: X_mb, self.Y_tr:Y_mb})
              
                # generate Sample Imgs
                sampleImgsOfX2Y, sampleImgsOfY2X = sess.run([self.X2Y, self.Y2X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # 結果をappend
                self.loss["d_loss_X"].append(d_loss_X_value)
                self.loss["d_loss_Y"].append(d_loss_Y_value)
                self.loss["g_loss"].append(g_loss_value)
                print("epoch:" + str(epoch))

                # lossの可視化
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(self.loss, epoch)

                # 画像の変換テスト
                if epoch % self.epoch_saveSampleImg == 0:
                    save_imgs(sampleImgsOfX2Y, epoch=str(epoch)+"X2Y")
                    save_imgs(sampleImgsOfY2X, epoch=str(epoch)+"Y2X")

                # parameterのsave
                if epoch % self.epoch_saveParamter == 0:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")


        
    
    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

# main func

In [ ]:
if __name__ == "__main__":
    CGAN = cycleGAN()
    CGAN.train()

init... name of g:X2Y
init... name of g:Y2X
init... name of d:X2Y
init... name of d:Y2X
called... name of g:X2Y
called... name of g:Y2X
called... name of g:Y2X
called... name of g:X2Y
called... name of d:Y2X
called... name of d:Y2X
called... name of d:X2Y
called... name of d:X2Y
epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29
epoch:30
epoch:31
epoch:32
epoch:33
epoch:34
epoch:35
epoch:36
epoch:37
epoch:38
epoch:39
epoch:40
epoch:41
epoch:42
epoch:43
epoch:44
epoch:45
epoch:46
epoch:47
epoch:48
epoch:49
epoch:50
epoch:51
epoch:52
epoch:53
epoch:54
epoch:55
epoch:56
epoch:57
epoch:58
epoch:59
epoch:60
epoch:61
epoch:62
epoch:63
epoch:64
epoch:65
epoch:66
epoch:67
epoch:68
epoch:69
epoch:70
epoch:71
epoch:72
epoch:73
epoch:74
epoch:75
epoch:76
epoch:77
epoch:78
epoch:79
epoch:80
e